In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [42]:
import re

In [2]:
df1 = pd.read_csv('./data/20170622_1514_SV (376k).csv')

/Users/nicholashirons/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,6,9,10,11,22,25,28,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,55,56,57,58,59,60,61,62,63,64,67,68,69,70,71,72,73,74,75,76,77,78,80,81,84,85,86,87,88,89,90,93,94,95,96,97,98,99,100,101,102,103,104,105,110,111,114,115,116,117,118,119,120,121,131,132,133,136,138,139,141,151,158,168,173) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df2 = pd.read_csv('./data/SVC Aggregate (115k).csv')

/Users/nicholashirons/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,25,29,35,40,43,44,48,53,54,55,56,59,60,61,63,64,67,68,69,73,75,76,77,114,115,116,117,118,119,120,121,133,153,154,155,156,176) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Preliminary exploration

In [60]:
df1.shape[0]

376970

In [61]:
print('df1 has {} rows and {} columns'.format(*df1.shape))
print('df2 has {} rows and {} columns'.format(*df2.shape))

df1 has 376970 rows and 176 columns
df2 has 185161 rows and 178 columns


In [62]:
print('The column(s) in df2 but not df1 are {}.'.format([col for col in df2.columns if col not in df1.columns]))
print('The column(s) in df1 but not df2 are {}.'.format([col for col in df1.columns if col not in df2.columns]))

The column(s) in df2 but not df1 are ['State', 'Zip', 'CASS_STD_STATE'].
The column(s) in df1 but not df2 are ['Unnamed: 175'].


In [16]:
df1.columns

Index(['File Type', 'DV Dealer ID', 'Vendor Dealer ID', 'DMS Type',
       'RO Number', 'Open Date', 'Close Date', 'Service Advisor Number',
       'Service Advisor Name', 'RO Department',
       ...
       'CASS_STD_FIPS', 'CASS_STD_EWS', 'CASS_STD_LACS', 'CASS_STD_ZIPMOV',
       'CASS_STD_Z4LOM', 'CASS_STD_NDIAPT', 'CASS_STD_NDIRR',
       'CASS_STD_LACSRT', 'CASS_STD_ERROR_CD', 'Unnamed: 175'],
      dtype='object', length=176)

In [24]:
df1.iloc[:,:10].head()

,File Type,DV Dealer ID,Vendor Dealer ID,DMS Type,RO Number,Open Date,Close Date,Service Advisor Number,Service Advisor Name,RO Department
0,Service,DVD03865,0013900001YPjh8AAD,ADP,410202,10/4/10,12/3/10,269.0,"CASHMAN,PATRICK M",S
1,Service,DVD03865,0013900001YPjh8AAD,ADP,411751,10/14/10,1/18/11,205.0,"CONKLIN,DEAN",S
2,Service,DVD03865,0013900001YPjh8AAD,ADP,413079,10/23/10,11/26/10,269.0,"CASHMAN,PATRICK M",S
3,Service,DVD03865,0013900001YPjh8AAD,ADP,413201,10/25/10,11/29/10,207.0,NaN,S
4,Service,DVD03865,0013900001YPjh8AAD,ADP,414116,11/1/10,11/30/10,201.0,"MYSLINSKI,DEAN",S


# Exploring column types

In [63]:
df1.dtypes.value_counts()

object     129
float64     45
int64        2
dtype: int64

In [64]:
object_cols = [col for col in df1.columns if df1[col].dtype == 'O']
float_cols = [col for col in df1.columns if df1[col].dtype == 'float64']
int_cols = [col for col in df1.columns if df1[col].dtype == 'int64']

In [66]:
# Check counts
len(object_cols) + len(float_cols) + len(int_cols) == len(df1.columns)

True

## Object types

In [74]:
# Examine object data types
pd.Series([type(df1[col][0]) for col in object_cols]).value_counts()

<class 'str'>      85
<class 'float'>    41
<class 'int'>       3
dtype: int64

In [100]:
obj_string_cols = [col for col in object_cols if type(df1[col][0]) == str]
obj_int_cols = [col for col in object_cols if type(df1[col][0]) == int]
obj_float_cols = [col for col in object_cols if type(df1[col][0]) == float]

In [104]:
print('There are {} object columns that contain a string in first entry.'.format(len(obj_string_cols)))
print('There are {} object columns that contain an integer in first entry.'.format(len(obj_int_cols)))
print('There are {} object columns that contain a float in first entry.'.format(len(obj_float_cols)))

There are 85 object columns that contain a string in first entry.
There are 3 object columns that contain an integer in first entry.
There are 41 object columns that contain a float in first entry.


## Idenify "long" strings (len > 20)

In [125]:
long_strings = sorted([col for col in obj_string_cols if len(df1[col][0]) > 20], reverse = True)

In [124]:
long_strings_and_len = sorted([(col,len(df1[col][0])) for col in obj_string_cols if len(df1[col][0]) > 20],
                              key = lambda s: s[1],
                              reverse = True)

In [128]:
long_strings_and_len

[('Operation Code Descriptions', 302),
 ('Labor Cause', 141),
 ('Tech Name', 130),
 ('Operation Line Sale', 69),
 ('Operation Line Cost', 67),
 ('Operation Codes', 59),
 ('Labor Bill Hours', 58),
 ('Parts Sale', 57),
 ('Labor Tech Hours', 54),
 ('Labor Sale', 52),
 ('Labor Cost', 49),
 ('Operation Sale Types', 45),
 ('Tech Number', 34),
 ('Misc Cost', 27),
 ('Misc Sale', 27),
 ('Tech Labor Line', 23),
 ('Operation Line Number', 23)]

In [132]:
df1[long_strings].head()

,Tech Number,Tech Name,Tech Labor Line,Parts Sale,Operation Sale Types,Operation Line Sale,Operation Line Number,Operation Line Cost,Operation Codes,Operation Code Descriptions,Misc Sale,Misc Cost,Labor Tech Hours,Labor Sale,Labor Cost,Labor Cause,Labor Bill Hours
0,126|||126|126|126|126||126|126|126,"O'BRIEN,CHARLES|||O'BRIEN,CHARLES|O'BRIEN,CHAR...",1|2|3|4|5|6|7|8|9|10|11,5401.32|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.0...,WESP|WESP|WESP|INS|INS|INS|INS|WESP|INS|CF|CF,7508.82|325.00|175.00|0.00|0.00|0.00|0.00|27.7...,1|2|3|4|5|6|7|8|9|10|11,4520.58|275.00|175.00|0.00|0.00|0.00|0.00|27.7...,L001|9999|9997|99P|GBATT|GTERM|GTIRE|9997|NWD|...,"REPLACE 8 INJ, HIGH ANDLOW PRESS FUEL PUMPS, F...",|325.00|175.00|||||27.74|||,|275.00|175.00|||||27.74|||,0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0...,2107.50|||0.00|0.00|0.00|0.00||0.00|1829.10|-1...,662.50|||0.00|0.00|0.00|0.00||0.00|532.65|-532.65,VISAUL INSPECTION FUEL TANK FULL OF WATER AND ...,25.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|...
1,126|||126|125||126,"O'BRIEN,CHARLES|||O'BRIEN,CHARLES|KEITH LINEBE...",1|2|3|4|5|6|7,1046.56|0.00|0.00|0.00|0.00|0.00|205.20,CFT|CFT|CFT|INS|CMTR|CMTR|CFT,1112.36|320.00|25.09|0.00|12.75|0.85|342.38,1|2|3|4|5|6|7,564.43|300.00|25.09|0.00|5.40|0.85|163.97,T6|9999|9997|99P|NCSI|9997|G15,PERFOMED ENGINEPERFORMANCE DIAG LOSINGFUEL PRE...,|320.00|25.09|||0.85|,|300.00|25.09|||0.85|,0.07|0.00|0.00|0.00|0.00|0.00|0.00,65.80|||0.00|12.75||137.18,15.90|||0.00|5.40||39.75,|||,0.60|0.00|0.00|0.00|0.30|0.00|1.50
2,152,"GREGORY,BEDFORD",1,0,INS,0,1,0,NWD,NWD,NaN,NaN,0,0,0,NaN,0
3,199|199|199|199,"DEFAULT,TECH|DEFAULT,TECH|DEFAULT,TECH|DEFAULT...",1|2|3|4,32.74|0.00|0.00|42.55,CMT|INS|CF|CF,32.74|0.00|0.00|42.55,1|2|3|4,12.81|0.00|0.00|42.55,NC|99P|NC|NC,NO CHARGE|PERFORM MULTI-POINTINSPECTION|NO CHA...,|||,|||,0.00|0.00|0.00|0.00,0.00|0.00|0.00|0.00,0.00|0.00|0.00|0.00,|||,0.00|0.00|0.00|0.00
4,199,"DEFAULT,TECH",1,564.99,INS,564.99,1,470.88,NC,SHOP NOV,NaN,NaN,0,0,0,NaN,0


In [136]:
df1['Operation Code Descriptions'].str.split('|')[0]

['REPLACE 8 INJ, HIGH ANDLOW PRESS FUEL PUMPS, FULLINES AND RAILS, RELINEFUEL TANK, FLUSH FUELSYSTEM, LOF, NEW COOLANT',
 '',
 '',
 'PERFORM MULTI-POINTINSPECTION',
 'BATTERY TEST PERFORMEDAND BATTERY OK ON THISVISIT',
 'BATTERY TERMINALS GOOD ATTHIS TIME',
 'TIRES INSPECTED AND OK ONTHIS VISIT.',
 '',
 'SEE LINE A',
 'DRAW TIME',
 'DRAW TIME']

### Notice how some words (likely last / first words of consecutive written lines) are mistakenly merged together. This will cause inaccuracy for any later NLP.

In [221]:
import importlib

In [222]:
importlib.reload(word_parser)

<module 'word_parser' from '/Users/nicholashirons/carforce/word_parser.py'>

In [223]:
from word_parser import infer_spaces
from word_parser import separate_merged_words

In [210]:
infer_spaces('BATTERYTESTPERFORMEDANDBATTERYOKONTHISVISIT'.lower())

'battery test performed and battery ok on this visit'

In [211]:
test_string = df1['Operation Code Descriptions'].str.split('|')[0][0]

In [212]:
test_string

'REPLACE 8 INJ, HIGH ANDLOW PRESS FUEL PUMPS, FULLINES AND RAILS, RELINEFUEL TANK, FLUSH FUELSYSTEM, LOF, NEW COOLANT'

In [229]:
separate_merged_words(test_string)

'replace 8 i n j, high and low press fuel pumps, full in es and rails, re line fuel tank, flush fuel system, l of, new coolant'

In [228]:
test2 = df1['Operation Code Descriptions'].str.split('|')[0][6]

In [230]:
separate_merged_words(test2)

'tires inspected and ok on this visit .'